檢查有無安裝 boto3

```
!pip list | grep boto
```

沒有的話就 install 一下。

```
pip install boto3
```

## list objects in the bucket


In [139]:
import boto3
import re

In [140]:
# confige
s3_access_key = 'AKIAVF********5W5XB'
s3_secret_key = '8K1oUcrO*************kMEBpA4M'
name_bucket = 'crawl-voice'

In [141]:
s3 = boto3.client('s3',
                  aws_access_key_id=s3_access_key,
                  aws_secret_access_key=s3_secret_key)

In [142]:
bkts = s3.list_buckets()
bkts['Buckets']

[{'Name': 'crawl-voice',
  'CreationDate': datetime.datetime(2021, 7, 12, 15, 47, 45, tzinfo=tzutc())},
 {'Name': 'sagemaker-us-west-2-355444812467',
  'CreationDate': datetime.datetime(2021, 7, 2, 14, 46, 37, tzinfo=tzutc())}]

In [145]:
# list objects
objs = s3.list_objects_v2(Bucket = name_bucket, MaxKeys=3)
[x['Key'] for x in objs['Contents']]

['category_00/', 'category_00/s01-000.wav', 'category_00/s01-001.wav']

In [146]:
# list all object
objs = []
tmp_objs = s3.list_objects_v2(Bucket = name_bucket, MaxKeys=1000)
objs = objs + [x['Key'] for x in tmp_objs['Contents']]
while 'NextContinuationToken' in tmp_objs.keys():
    tmp_objs = s3.list_objects_v2(Bucket = name_bucket, MaxKeys=1000, ContinuationToken = tmp_objs['NextContinuationToken'])
    objs = objs + [x['Key'] for x in tmp_objs['Contents']]
    
len(objs)

5372

In [147]:
# filter all wav files
p = re.compile('.+\wav')
objs_filter = [ x for x in objs if p.match(x) ]

len(objs_filter)

5345

In [148]:
folder = {}
for i in objs_filter:
    tmp_re = re.search(pattern = 'category_[0-9]{2}', string = i)
    fname = tmp_re.group()
    tmp_url = 'https://crawl-voice.s3.us-east-1.amazonaws.com/'+i
    if fname not in folder.keys():
        folder[fname] = [tmp_url]
    else:
        folder[fname] = folder[fname] + [tmp_url]

In [149]:
folder.keys()

dict_keys(['category_00', 'category_01', 'category_02', 'category_04', 'category_05', 'category_06', 'category_07', 'category_09'])

In [ ]:
with open('filename.pickle', 'wb') as handle:
    pickle.dump(folder, handle, protocol=pickle.HIGHEST_PROTOCOL)

---

## 柏龍作法 

from local file

In [90]:
import os
import pickle

In [137]:
folder = {}

for i in range(10):
    i = '0'+str(i)
    tmp = f'category_{i}'
    folder[tmp] = ['https://crawl-voice.s3.us-east-1.amazonaws.com/'+tmp+'/'+x for x in os.listdir(f'../data/category_{i}')]

In [138]:
folder.keys()

dict_keys(['category_00', 'category_01', 'category_02', 'category_03', 'category_04', 'category_05', 'category_06', 'category_07', 'category_08', 'category_09'])

In [31]:
with open('filename.pickle', 'wb') as handle:
    pickle.dump(folder, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)
# print(a == b)

---

## END